## 1. System Information and GPU Detection

In [ ]:
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"\nCUDA (ROCm) available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("No GPU detected - running on CPU")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

## 2. Import Required Libraries

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

print("Libraries imported successfully")

## 3. Create Synthetic Dataset

We'll create a simple binary classification dataset with 2 features.

In [ ]:
class SyntheticDataset(Dataset):
    def __init__(self, n_samples=1000, n_features=2, random_state=42):
        np.random.seed(random_state)
        
        # Generate synthetic data for binary classification
        # Class 0: centered around (-1, -1)
        # Class 1: centered around (1, 1)
        class_0 = np.random.randn(n_samples // 2, n_features) + np.array([-1, -1])
        class_1 = np.random.randn(n_samples // 2, n_features) + np.array([1, 1])
        
        self.X = np.vstack([class_0, class_1]).astype(np.float32)
        self.y = np.hstack([np.zeros(n_samples // 2), np.ones(n_samples // 2)]).astype(np.float32)
        
        # Shuffle the data
        indices = np.random.permutation(n_samples)
        self.X = self.X[indices]
        self.y = self.y[indices]
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

# Create dataset
dataset = SyntheticDataset(n_samples=1000, n_features=2)
print(f"Dataset created with {len(dataset)} samples")

# Visualize the data
X_numpy = dataset.X
y_numpy = dataset.y

plt.figure(figsize=(8, 6))
plt.scatter(X_numpy[y_numpy == 0][:, 0], X_numpy[y_numpy == 0][:, 1], 
           c='blue', label='Class 0', alpha=0.5)
plt.scatter(X_numpy[y_numpy == 1][:, 0], X_numpy[y_numpy == 1][:, 1], 
           c='red', label='Class 1', alpha=0.5)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Synthetic Binary Classification Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Split Data and Create DataLoaders

In [ ]:
# Split dataset into train (70%), validation (15%), and test (15%)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Batch size: {batch_size}")

## 5. Define Sequential Neural Network

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=2, hidden_size=16, output_size=1):
        super(SimpleNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

# Initialize model
model = SimpleNN(input_size=2, hidden_size=16, output_size=1).to(device)
print(model)
print(f"\nModel has {sum(p.numel() for p in model.parameters())} parameters")
print(f"Model is on device: {next(model.parameters()).device}")

## 6. Define Loss Function and Optimizer

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print(f"Loss function: {criterion}")
print(f"Optimizer: {optimizer}")

## 7. Training Function

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1)  # Add dimension for BCELoss
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        predictions = (outputs > 0.5).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

print("Training function defined")

## 8. Evaluation Function

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.unsqueeze(1)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            predictions = (outputs > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

print("Evaluation function defined")

## 9. Train the Model

In [ ]:
num_epochs = 50
train_losses = []
train_accs = []
val_losses = []
val_accs = []

print("Starting training...\n")

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validate
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}\n")

print("Training complete!")

## 10. Plot Training History

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot loss
ax1.plot(train_losses, label='Train Loss', linewidth=2)
ax1.plot(val_losses, label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot accuracy
ax2.plot(train_accs, label='Train Accuracy', linewidth=2)
ax2.plot(val_accs, label='Validation Accuracy', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Test the Model

In [ ]:
# Evaluate on test set
test_loss, test_acc = evaluate(model, test_loader, criterion, device)

print("\n" + "="*50)
print("TEST RESULTS")
print("="*50)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("="*50)

## 12. Detailed Test Metrics

In [ ]:
# Get predictions for all test samples
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        predictions = (outputs > 0.5).float().cpu().numpy()
        
        all_predictions.extend(predictions)
        all_labels.extend(labels.numpy())

all_predictions = np.array(all_predictions).flatten()
all_labels = np.array(all_labels)

# Print classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, 
                          target_names=['Class 0', 'Class 1'],
                          digits=4))

## 13. Visualize Decision Boundary

In [ ]:
# Create a mesh grid
x_min, x_max = X_numpy[:, 0].min() - 1, X_numpy[:, 0].max() + 1
y_min, y_max = X_numpy[:, 1].min() - 1, X_numpy[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

# Predict on mesh grid
model.eval()
with torch.no_grad():
    grid_points = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32).to(device)
    Z = model(grid_points).cpu().numpy()
    Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, levels=20, cmap='RdYlBu', alpha=0.6)
plt.colorbar(label='Predicted Probability')
plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)

# Plot data points
plt.scatter(X_numpy[y_numpy == 0][:, 0], X_numpy[y_numpy == 0][:, 1], 
           c='blue', label='Class 0', alpha=0.7, edgecolors='black')
plt.scatter(X_numpy[y_numpy == 1][:, 0], X_numpy[y_numpy == 1][:, 1], 
           c='red', label='Class 1', alpha=0.7, edgecolors='black')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Neural Network Decision Boundary')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Summary

This notebook demonstrated:
- ✅ GPU/CUDA (ROCm) detection and device information
- ✅ Creating a custom PyTorch Dataset with synthetic data
- ✅ Using DataLoader for efficient batch processing
- ✅ Building a sequential neural network
- ✅ Training loop with proper train/validation split
- ✅ Model evaluation on test set
- ✅ Visualization of results and decision boundaries

The model successfully learned to classify the synthetic binary dataset with high accuracy!